# 'size'가 8 이하인 경우만 남기기 + 페이지넘버 가리키는 폰트 제외 하고 + 왼쪽부터 항상 읽히도록

In [ ]:
import fitz

pdf_file = "duden.pdf"
doc = fitz.open(pdf_file)

refined_spans = []

for page in doc[31:]:
    margin = 190.0 
    left_col = []
    right_col = []

    for idx, text in enumerate(page.get_text("dict")["blocks"]):
        if "lines" in text:
            for line in text["lines"]:
                for span in line["spans"]:
                    if span["size"] <= 8 and span["font"] != "Po.FrutigerCn-mager":  # 필요 없는 거 제거
                        refined = {
                            "page": idx,
                            "text": span["text"],
                            "size": span["size"],
                            "font": span["font"],
                            "color": span["color"],
                            "origin": span["origin"],
                        }

                        if refined["origin"][0] < margin:  
                            left_col.append(refined)  # 왼쪽에 위치한 텍스트
                        else: 
                            right_col.append(refined)  # 오른쪽에 위치한 텍스트


    # ✅ left 그룹을 먼저 저장하고, 그다음 right 그룹을 저장
    refined_spans.extend(sorted(left_col, key=lambda x: x["origin"][1]))   # y축 순서 유지
    refined_spans.extend(sorted(right_col, key=lambda x: x["origin"][1]))  # y축 순서 유지

print(len(refined_spans))

In [ ]:
refined_spans[50:200]

In [ ]:
# refined_spans에 저장된 text만 출력
for span in refined_spans:
    print(span["text"])

# 본격적으로 정제!

# NEW!
예문 분리 해야함

In [ ]:
import re

# 결과를 저장할 리스트
data = []

# 현재 row의 항목들
current_redewendung = []  # 관용어 텍스트들을 저장 (여러 스팬을 합쳐 하나의 관용어로 만듦)
current_bedeutung = []    # 의미(들)
current_beispiel = []     # 예시(들) → 여러 예시가 있을 수 있으므로 리스트로 관리

# 현재 상태: "redewendung", "bedeutung", "beispiel" 중 하나
state = None

# skip_mode: font가 "Po.Kepler-normal"이고 color==44527인 경우, 
#             redewendung 조건이 나오기 전까지 해당 span들을 무시
skip_mode = False

for span in refined_spans:
    size = span["size"]
    font = span["font"]
    color = span["color"]
    text = span["text"].strip()  # 앞뒤 공백 제거

    # 현재 span에 "〈lat.〉" 또는 "〈franz.〉"가 포함되어 있다면,
    # 현재 진행 중인 redewendung 그룹을 종료(저장)한 후 language_skip을 활성화하고 이 span은 저장하지 않습니다.
    if "〈lat.〉" in text or "〈franz.〉" in text:
        if current_redewendung:
            data.append({
                "redewendung": " ".join(current_redewendung).strip(),
                "bedeutung": " ".join(current_bedeutung).strip() if current_bedeutung else None,
                "beispiel": current_beispiel if current_beispiel else None
            })
        current_redewendung = []
        current_bedeutung = []
        current_beispiel = []
        language_skip = True
        continue
    
    # --- skip_mode 처리 ---
    if skip_mode:
        # redewendung 조건이 나오면 skip_mode 해제
        if size == 7.5 and font == "Po.FrutigerCn-fett" and color == 44527:
            skip_mode = False
        else:
            continue

    # 스킵 모드 시작: 예를 들어 "♦" 같은 표시가 있을 때 (font가 "Po.Kepler-normal"이고 color가 44527)
    if (font == "Po.Kepler-normal" and color == 44527) or (font == "Po.FrutigerCn-extrafett" and color == 2236191):
        skip_mode = True
        continue

    # --- redewendung 조건 ---
    if size == 7.5 and font == "Po.FrutigerCn-fett" and color == 44527:
        # 만약 현재 redewendung 상태인데 아직 의미(또는 예시)가 시작되지 않았다면,
        # 연속되는 redewendung 스팬은 같은 idiom으로 취급하여 합칩니다.
        if current_redewendung and state == "redewendung" and not current_bedeutung:
            current_redewendung.append(text)
        else:
            # 이미 의미나 예시가 시작된 상태라면, 새 관용어의 시작으로 취급하고 row를 분리합니다.
            if current_redewendung:
                data.append({
                    "redewendung": " ".join(current_redewendung),
                    "bedeutung": " ".join(current_bedeutung) if current_bedeutung else None,
                    "beispiel": current_beispiel if current_beispiel else None
                })
            # 새로운 관용어 row 시작 (새 row에서는 의미와 예시는 초기화)
            current_redewendung = [text]
            current_bedeutung = []
            current_beispiel = []
        state = "redewendung"
        continue

    # --- 의미(=bedeutung) 처리 ---
    # 1) 새 의미 그룹의 시작 marker: (size 7.8, color 2236191, font "Po.Kepler-fett") 
    #    그리고 텍스트가 "1.", "2.", "3.", "4." (공백 제거 후)인 경우
    if color == 2236191 and font == "Po.Kepler-fett":
        if text in ["1.", "2.", "3.", "4."]:
            # 이미 의미나 예시가 기록되어 있다면 같은 관용어 내에서 새 의미 그룹으로 분리
            if current_bedeutung or current_beispiel:
                data.append({
                    "redewendung": " ".join(current_redewendung),
                    "bedeutung": " ".join(current_bedeutung) if current_bedeutung else None,
                    "beispiel": current_beispiel if current_beispiel else None
                })
                # 관용어(redewendung)는 그대로 유지하고 의미와 예시는 초기화
                current_bedeutung = []
                current_beispiel = []
            current_bedeutung.append(text)
            state = "bedeutung"
            continue
    # 2) 일반적인 의미 텍스트: (size 7.8, font "Po.Kepler-kursiv", color 2236191)
    if font == "Po.Kepler-kursiv" and color == 2236191:
        current_bedeutung.append(text)
        state = "bedeutung"
        continue

    # --- beispiel(예시) 처리 ---
    # 일반적으로 font "Po.Kepler-normal" & color 2236191이면 예시로 취급함
    if font == "Po.Kepler-normal" and color == 2236191:
        # 단, 아직 의미 그룹이 시작되지 않은 상태이고 현재 state가 redewendung라면,
        # 이것이 관용어 중간에 끼어있는 "이상한" 스팬일 수 있으므로 redewendung에 합칩니다.
        if state == "redewendung" and not current_bedeutung:
            continue
        else:
            # 새 조건 추가: 만약 스팬텍스트 원본이 (앞에 공백 후) 대문자로 시작하면 새로운 예시 항목으로 간주
            if re.match(r'^\s+[A-Z]', span["text"]):
                # 새로운 예시 항목을 추가 (양쪽 공백 제거)
                current_beispiel.append(text)
            else:
                # 이미 예시 항목이 있다면, 마지막 예시 항목에 이어서 붙임
                if current_beispiel:
                    current_beispiel[-1] += " " + text
                else:
                    current_beispiel.append(text)
            state = "beispiel"
        continue

    # --- 그 외 텍스트 ---
    # 별도로 분류되지 않은 텍스트는 현재 활성 state에 따라 추가합니다.
    if state == "redewendung":
        current_redewendung.append(text)
    elif state == "bedeutung":
        current_bedeutung.append(text)
    elif state == "beispiel":
        # 만약 여기서도 새 조건에 해당하면 새로운 예시 항목으로 추가
        if re.match(r'^\s+[A-Z]', span["text"]):
            current_beispiel.append(text)
        else:
            if current_beispiel:
                current_beispiel[-1] += " " + text
            else:
                current_beispiel.append(text)

# 반복문 종료 후, 남아있는 row 저장
if current_redewendung:
    data.append({
        "redewendung": " ".join(current_redewendung),
        "bedeutung": " ".join(current_bedeutung).strip() if current_bedeutung else None,
        "beispiel": current_beispiel if current_beispiel else None
    })

print(data)

In [3]:
import pandas as pd
df = pd.DataFrame(data)
import ace_tools_open as tools
#tools.display_dataframe_to_user(name="Redewendung Data", dataframe=df)

In [5]:
df = pd.DataFrame(data)
#df[:10]

In [ ]:
len(data)

In [ ]:
import csv
csv_file = "german-1.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["Redewendung", "Bedeutung", "Beispiel"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for item in data:
        writer.writerow({
            "Redewendung": item["redewendung"],
            "Bedeutung": item["bedeutung"],
            "Beispiel": item["beispiel"]
        })

print(f"\n총 {len(data)}개의 항목이 '{csv_file}' 파일에 저장되었습니다.")


"- " 없애기

In [8]:
import pandas as pd

df = pd.read_csv("german-1.csv")

# 문자열이나 리스트에 대해 "- "를 제거하는 함수
def fix_hyphenation(text):
    if isinstance(text, list):
        # 리스트 내부의 각 요소에 대해 재귀적으로 처리합니다.
        return [fix_hyphenation(item) for item in text]
    elif isinstance(text, str):
        return text.replace("- ", "")
    return text  # 문자열도 리스트도 아닌 경우 그대로 반환

# 데이터프레임의 모든 셀에 대해 함수를 적용합니다.
df = df.applymap(fix_hyphenation)

()없애기 다만 그룹화하기 (umgangs.) + 화살표랑 : 기호 지우기

In [ ]:
import pandas as pd
import re
from collections import defaultdict
import json

######################################
# 기존 전처리 (괄호 제거 및 그룹화 처리)
######################################

# 그룹화할 딕셔너리
grouped_expressions = defaultdict(list)

# 괄호 안의 내용 제거 함수 + 그룹화 처리
def clean_redewendung(text):
    if isinstance(text, list):
        return [clean_redewendung(item) for item in text]
    if isinstance(text, str):
        match = re.search(r"\((.*?)\)", text)  # 괄호 안의 내용 찾기
        if match:
            annotation = match.group(1)
            grouped_expressions[annotation].append(text)
            text = re.sub(r"\s*\(.*?\)", "", text)  # 괄호 및 그 안의 내용 삭제
        return text.strip()
    return text

# "Beispiel" 칼럼의 경우, 문자열 끝에 붙은 괄호 그룹만 제거하는 함수
def remove_trailing_parentheses(text):
    if isinstance(text, list):
        return [remove_trailing_parentheses(item) for item in text]
    if isinstance(text, str):
        # 문자열의 끝에 위치한 괄호와 그 안의 내용을 제거 (예: " ... (내용)" → "...")
        return re.sub(r'\s*\([^)]*\)\s*$', '', text).strip()
    return text

######################################
# unwanted 기호 ("↑", ":") 제거 처리
######################################

def remove_unwanted_symbols(text):
    """
    주어진 문자열에서 "↑"와 ":" 기호를 제거합니다.
    """
    if isinstance(text, list):
        return [remove_unwanted_symbols(item) for item in text]
    if isinstance(text, str):
        # "↑"와 ":" 기호 제거
        cleaned = text.replace("↑", "").replace(":", "")
        # 불필요한 공백 정리
        return cleaned.strip()
    return text

######################################
# DataFrame 전처리 진행
######################################

# CSV 파일 읽기
df = pd.read_csv("german-1.csv")

# "Redewendung" 칼럼 전처리: 괄호 제거 및 그룹화 처리
df["Redewendung"] = df["Redewendung"].apply(clean_redewendung)

# "Beispiel" 칼럼 전처리: 문자열 끝에 붙은 괄호 그룹만 제거 (중간 괄호는 그대로 유지)
if "Beispiel" in df.columns:
    df["Beispiel"] = df["Beispiel"].apply(remove_trailing_parentheses)

# 각 칼럼에서 unwanted 기호 ("↑", ":") 제거 처리
df["Redewendung"] = df["Redewendung"].apply(remove_unwanted_symbols)
df["Beispiel"]    = df["Beispiel"].apply(remove_unwanted_symbols)
df["Bedeutung"]  = df["Bedeutung"].apply(remove_unwanted_symbols)

######################################
# 최종 DataFrame 및 그룹화된 표현 JSON 저장
######################################

# 수정된 CSV 파일 저장
df.to_csv("german-2.csv", index=False)
print("최종 CSV 파일 저장 완료: final_cleaned_filtered.csv")

# 그룹화된 표현(grouped_expressions) 저장 함수
def save_json(data, filename="grouped_expressions.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"JSON data saved to {filename}")

# JSON 저장 실행
save_json(grouped_expressions)

In [ ]:
# 빈 값을 판단하는 헬퍼 함수
def is_empty(value):
    if value is None:
        return True
    if isinstance(value, str):
        return value.strip() == ""  # 공백만 있는 경우도 빈 값으로 처리
    if isinstance(value, list):
        return len(value) == 0  # 빈 리스트라면 True
    return False

# 칼럼 중 하나라도 빈 row (빈 문자열, None 또는 빈 리스트)를 필터링하여 출력
print("\n빈 칼럼이 있는 row들:")
for idx, row in enumerate(data):
    # redewendung, bedeutung, beispiel 중 하나라도 값이 비어있으면 출력합니다.
    if is_empty(row.get("redewendung")) or is_empty(row.get("bedeutung")) or is_empty(row.get("beispiel")):
        print(f"Row {idx}: {row}")

# # 또는 리스트 내포를 사용해 필터링할 수도 있습니다.
# empty_rows = [
#     row for row in data
#     if is_empty(row.get("redewendung")) or is_empty(row.get("bedeutung")) or is_empty(row.get("beispiel"))
# ]
# print("\n빈 칼럼이 있는 row들 (리스트 내포):")
# for row in empty_rows:
#     print(row)

검토 후에 meaning 넘버 지울것임

In [ ]:
import pandas as pd
import re

# CSV 파일 경로
file_path = "german-2.csv"

try:
    # CSV 파일 읽기 (자동 구분자 감지, 문제 있는 줄 건너뛰기)
    df = pd.read_csv(file_path, sep=None, engine="python", on_bad_lines="skip")

    # 첫 번째 칼럼 이름 가져오기
    first_col = df.columns[0]
    second_col = df.columns[1] if len(df.columns) > 1 else None

    # 대괄호 내용 제거 함수 (문자열뿐만 아니라 리스트형 셀도 처리)
    def remove_brackets(text):
        if isinstance(text, list):
            return [remove_brackets(item) for item in text]
        if isinstance(text, str):
            # 정규표현식을 사용해 대괄호 [] 내의 모든 내용을 제거 후 양쪽 공백 제거
            return re.sub(r"\[.*?\]", "", text).strip()
        return text

    # 첫 번째 및 두 번째 칼럼에서 "["와 "]" 포함된 부분 제거
    df[first_col] = df[first_col].apply(remove_brackets)
    if second_col:
        df[second_col] = df[second_col].apply(remove_brackets)

    # "..." 문자열만 제거하는 함수
    def remove_ellipsis(text):
        if isinstance(text, list):
            return [remove_ellipsis(item) for item in text]
        if isinstance(text, str):
            # "..." 부분만 찾아서 제거
            return re.sub(r"\.\.\.", "", text)
        return text

    # 첫 번째 칼럼에서 "..." 부분만 제거
    df[first_col] = df[first_col].apply(remove_ellipsis)

    # 세 번째 칼럼이 존재하는 경우에도 "..." 부분만 제거
    if len(df.columns) > 2:
        third_col = df.columns[2]
        df[third_col] = df[third_col].apply(remove_ellipsis)
    
    # 수정된 CSV 파일 저장
    df.to_csv("german-3.csv", index=False)
    print("Processed file saved")

except Exception as e:
    print(f"Error processing CSV file: {e}")

In [3]:
import pandas as pd

file_path = "german-5.csv"

try:
    # CSV 파일 읽기 (자동 구분자 감지, 문제 있는 줄 건너뛰기)
    df = pd.read_csv(file_path, sep=None, engine="python", on_bad_lines="skip")

    # 첫 번째 칼럼 이름 가져오기
    first_col = df.columns[0]

    # "/" 또는 "["이 포함된 행 필터링
    #filtered_rows = df[df[first_col].astype(str).str.contains(r"[\/\[]", regex=True, na=False)]
    filtered_rows = df[df[first_col].astype(str).str.contains(r"[\/]", regex=True, na=False)]
    count = df[df[first_col].astype(str).str.contains(r"[\;]", regex=True, na=False)].shape[0]
    #print(filtered_rows)
    print(f"Number of rows containing '/' or '[' in the first column: {len(filtered_rows)}")
    print(count)

except Exception as e:
    print(f"Error reading CSV file: {e}")

Number of rows containing '/' or '[' in the first column: 2533
174


In [5]:
file_path = "german-5.csv" 
df = pd.read_csv(file_path)

# 첫 번째와 두 번째 컬럼명 가져오기
first_col = df.columns[0]
second_col = df.columns[1]

# "/"가 포함된 행 필터링 (첫 번째와 두 번째 컬럼 모두 포함하는 경우)
filtered_df = df[
    df[first_col].astype(str).str.contains("/", na=False) & 
    df[second_col].astype(str).str.contains("/", na=False)
]

# 결과 저장
output_path = "german-5_filtered_both.csv"
filtered_df.to_csv(output_path, index=False)

print(f'필터링된 데이터가 "{output_path}"로 저장되었습니다.')


필터링된 데이터가 "german-5_filtered_both.csv"로 저장되었습니다.


## >><< 제거

In [ ]:
import pandas as pd
import re

# 파일 읽기
file_path = "german-3.csv"
df = pd.read_csv(file_path, encoding="utf-8")

# 정규표현식 패턴: '«' 또는 '»'
pattern = r"[«»]"

# DataFrame의 모든 문자열에서 해당 패턴을 제거
df = df.astype(str).applymap(lambda x: re.sub(pattern, "", x))

# 결과를 새로운 파일로 저장
output_path = "german-3.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"특수 문자가 제거된 데이터가 {output_path} 파일로 저장되었습니다.")

예문 분리안된것들 처리 german-4

# 여기 자동화 로직 다시 생각해보기

In [1]:
import pandas as pd
import re
import ast  # 문자열을 리스트로 변환하기 위해 사용

# CSV 파일 읽기
file_path = "german-4.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 마지막 칼럼 이름 가져오기
last_col = df.columns[-1]

def process_last_list_element(text):
    """
    문자열 형식의 리스트를 처리합니다.
    - 리스트 내의 모든 문자열 요소에서 "(" 기호의 총 개수를 계산합니다.
    - 만약 "("가 2번 이상 등장하면 원본 문자열을 그대로 반환합니다.
    - 그렇지 않으면, 리스트의 마지막 요소에서 괄호와 그 안의 내용을 제거한 후
      수정된 리스트를 문자열로 변환하여 반환합니다.
    """
    if not isinstance(text, str):
        return text

    try:
        # 문자열이 리스트 형태로 되어있는 경우 실제 리스트로 변환
        list_data = ast.literal_eval(text)
        
        # 리스트이고 길이가 2 이상인 경우만 처리
        if isinstance(list_data, list) and len(list_data) >= 2:
            # 리스트의 모든 요소에서 "("의 총 개수를 셉니다.
            total_paren_count = sum(item.count("(") for item in list_data if isinstance(item, str))
            
            # "("가 2번 이상 등장하면 원본 문자열 그대로 반환
            if total_paren_count >= 2:
                print("두 개 이상의 괄호가 발견됨:", text)
                return text
            else:
                # 그렇지 않으면, 마지막 요소에서 괄호와 그 안의 내용을 제거
                last_item = list_data[-1]
                cleaned_last_item = re.sub(r"\([^)]*\)", "", last_item)
                if last_item != cleaned_last_item:
                    list_data[-1] = cleaned_last_item
                    # 디버깅 메시지 출력 (필요에 따라 주석 처리 가능)
                    # print(f"변경됨: {last_item} → {cleaned_last_item}")
                return str(list_data)
        else:
            return text
    except (ValueError, SyntaxError):
        # 변환 실패 시 원본 반환
        return text

# 마지막 칼럼에 위 함수를 적용
df[last_col] = df[last_col].astype(str).apply(process_last_list_element)

# 결과를 CSV 파일로 저장
output_path = "filtered_german-4.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"처리된 데이터가 {output_path} 파일로 저장되었습니다.")


두 개 이상의 괄호가 발견됨: ['Ich habe das Buch von A bis Z gelesen. Die Geschichte ist von A bis Z erfunden.', 'Keine Rolle, die er nicht von A bis Z umgeschrieben hätte (Hörzu 25, 1975, 18). Der komplexe, har- monische, reinsortige Wein mit Schmelz weiß von A bis Z zu begeistern und bietet viel Weingenuss für relativ wenig Geld (NZZ, 30. 5. 2004, 99).']
두 개 이상의 괄호가 발견됨: ['Die Polizisten hatten keine Ahnung, wen sie da verhaftet hatten.', 'So weht ein Hauch von Untergrund durch die Hotline-Gemeinde – Eingeweihten eröff- nen sich Welten, von denen der kommune Surfer keine blasse Ahnung hat (NZZ 21. 5. 1999). Okay, ich besorg dir welche, ver- sprach ich, dabei hatte ich nicht die lei- seste Ahnung, wo ich die Dinger herkrie- gen sollte (FR 8. 3. 1997, 4).']
두 개 이상의 괄호가 발견됨: ['Mit ihrem unbekümmerten Offen- sivfußball haben die Aufsteiger in der Bundesliga Akzente gesetzt.', 'Die neue Führung kann daran anknüpfen, wird aber, was Tempo und Entscheidungs- kraft angeht, noch mal eigene Akzente setzen 

In [4]:
file_path = "german-5.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 마지막 칼럼 이름 가져오기
last_col = df.columns[-1]

# 마지막 칼럼에서 '()' 괄호가 포함된 행 필터링
df_filtered = df[df[last_col].astype(str).str.contains(r"\(.*\)", regex=True, na=False)]

# 결과를 새로운 파일로 저장
output_path = "eg.csv"
df_filtered.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"'()' 괄호가 포함된 행이 {output_path} 파일로 저장되었습니다.")


'()' 괄호가 포함된 행이 eg.csv 파일로 저장되었습니다.


In [2]:
import pandas as pd
import re
import ast  # 문자열 형태의 리스트를 실제 리스트로 변환하기 위함

# CSV 파일 읽기
file_path = "filtered_german-4.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 마지막 칼럼 이름 가져오기
last_col = df.columns[-1]

# === 단계 1: 한 개의 예문에서 출처(괄호 안 내용) 제거 및 분리 함수 ===

def split_examples(text):
    """
    한 요소 내에 괄호로 감싼 출처가 2번 이상 등장하면, 
    각 출처 앞까지의 문장을 추출하여 예문들을 분리합니다.
    
    예:
    "Der Volkspapst ... Anfassen (Stern ...). Die Spanier wünschen sich ... Anfassen (Hörzu ...). Ende, Finger, ..."
    → 
    [
      "Der Volkspapst , der mit den Leuten singt, der in der Menge badet ein Papst zum Anfassen",
      "Die Spanier wünschen sich einen Star zum Anfassen und nicht nur einen, der gut Fußball spielt"
    ]
    """
    # 패턴: 괄호로 감싼 부분을 기준으로 앞쪽 텍스트를 추출
    pattern = re.compile(r'(.*?)\s*\([^)]*\)')
    results = []
    for match in pattern.finditer(text):
        part = match.group(1).strip()
        # 만약 앞에 불필요한 점이나 쉼표 등이 있다면 제거
        part = re.sub(r'^[\.\,\s]+', '', part)
        if part:
            results.append(part)
    return results

def process_example_element(text):
    """
    한 예문(문자열)을 처리합니다.
    - 만약 괄호 '('가 2번 이상 등장하면 split_examples()를 사용하여 여러 예문으로 분리합니다.
    - 그렇지 않으면, 단순히 모든 괄호와 그 안의 내용을 제거합니다.
    """
    if not isinstance(text, str):
        return [text]
    
    # 괄호 등장 횟수 확인
    count = text.count("(")
    if count >= 2:
        splitted = split_examples(text)
        if splitted:
            return splitted
        else:
            return [text]
    else:
        # 괄호가 1번 이하이면 모두 제거한 후 반환
        cleaned = re.sub(r'\s*\([^)]*\)', '', text).strip()
        return [cleaned]

# === 단계 2: 마지막 칼럼의 문자열(리스트 형태)을 처리하는 함수 ===

def process_last_list(text):
    """
    마지막 칼럼의 값은 리스트 형태의 문자열입니다.
    이를 실제 리스트로 변환한 후, 각 요소마다 process_example_element()를 적용하여
    (필요 시) 여러 예문으로 분리한 뒤, 최종 결과를 하나의 리스트로 만들어 문자열로 반환합니다.
    """
    if not isinstance(text, str):
        return text
    try:
        list_data = ast.literal_eval(text)
        if not isinstance(list_data, list):
            return text
        new_list = []
        for elem in list_data:
            # 각 요소를 처리하여 리스트로 반환된 예문들을 확장(평탄화)하여 new_list에 추가
            new_list.extend(process_example_element(elem))
        return str(new_list)
    except Exception as e:
        print(f"변환 실패 ({e}): {text}")
        return text

# === 단계 3: DataFrame의 마지막 칼럼 처리 및 저장 ===

# 마지막 칼럼의 모든 값에 대해 process_last_list() 적용
df[last_col] = df[last_col].astype(str).apply(process_last_list)

# 결과를 CSV 파일로 저장
output_path = "filtered_german-4-2.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"예문 분리 및 출처 제거 전처리가 완료되어 {output_path} 파일로 저장되었습니다.")

변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e2a7aa250>): nan
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e63478c40>): nan
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e63478700>): nan
변환 실패 (invalid syntax (<unknown>, line 1)): [Anfang vierzig und keinen Lebensgefähr- ten; sieht so aus, als hätte sie über ihre Karriere den Anschluss verpasst.']
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e289ee340>): nan
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e2a7aab80>): nan
변환 실패 (invalid syntax (<unknown>, line 1)): [Vielen Eltern, die schon ein Kind haben, gehen die Augen über, wenn das nächste kommt Das ältere wächst über Nacht (Presse 21. 12. 2013, 30).']
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e634acdc0>): nan
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e634acac0>): nan
변환 실패 (malformed node or string: <_ast.Name object at 0x7f0e2a79be50>): nan
변환 실패 (malformed node or string: <_ast.N

- 약어 교체
- 띄어쓰기 두번
- 이상한 괄호 (ugs) 삭제
- "- " 이런거 삭제
- 움라우트
- "-  " 이런것도 있고

stichwort 제거

In [15]:
import pandas as pd
import re
import ast  # 문자열 형태의 리스트를 실제 리스트로 변환

# CSV 파일 읽기
file_path = "german-5.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

def remove_trailing_strichworte(text):
    """
    주어진 문자열(text)의 맨 끝에, 쉼표와 공백으로 구분된 단어들의 목록이
    마침표로 끝난 부분이 있다면 이를 제거하고, 나머지 문장만 반환합니다.
    
    예시:
      입력: "Der Text endet hier. Borg, Dauer, einmal, Hieb, Lepschi, neu, Nimmerwiedersehen, Probe, Sitz."
      출력: "Der Text endet hier"
    
    만약 해당 패턴이 발견되지 않으면 원본 문자열을 그대로 반환합니다.
    """
    # 오른쪽에서 마지막 마침표('.')를 기준으로 분리
    parts = text.rsplit('.', 1)
    if len(parts) == 2:
        last_segment = parts[1].strip()
        # last_segment가 쉼표와 공백으로 구분된 단어들(알파벳, 숫자, 하이픈 등)만으로 구성되어 있으면 제거
        if last_segment and re.fullmatch(r'([A-Za-zÄÖÜäöüß0-9\-]+(?:,\s*[A-Za-zÄÖÜäöüß0-9\-]+)*)', last_segment):
            return parts[0].strip()
    return text

def process_last_cell(cell_text):
    """
    마지막 칼럼의 셀(cell_text)은 문자열 형태의 리스트입니다.
    이를 실제 리스트로 변환한 후, 리스트 내의 각 엘리먼트를 개별적으로 검사하여
    remove_trailing_strichworte() 함수를 적용합니다.
    
    처리된 리스트를 문자열 형태로 다시 반환합니다.
    """
    try:
        lst = ast.literal_eval(cell_text)
        if isinstance(lst, list):
            processed_list = []
            for elem in lst:
                if isinstance(elem, str):
                    processed_elem = remove_trailing_strichworte(elem)
                    processed_list.append(processed_elem)
                else:
                    processed_list.append(elem)
            return str(processed_list)
        else:
            return cell_text
    except Exception as e:
        print(f"변환 실패: {cell_text} (오류: {e})")
        return cell_text

# --- 마지막 칼럼에 process_last_cell 함수 적용 ---
last_col = df.columns[-1]
df[last_col] = df[last_col].astype(str).apply(process_last_cell)

# # --- 2. 모든 칼럼에서 대괄호([])가 포함된 셀이 있는지 확인 (예시로 첫 두 칼럼만) ---
# print("각 칼럼에서 대괄호([])를 포함하는 셀의 개수:")
# for col in df.columns[:2]:
#     count = df[col].astype(str).str.contains(r'\[.*?\]').sum()
#     print(f" - '{col}': {count} 개의 셀")

# # --- 3. 마지막 칼럼에서 대괄호([])를 포함하는 셀들 개별 엘리먼트별로 출력 ---
# # 마지막 칼럼 이름 가져오기
# last_col = df.columns[-1]

# print("\n마지막 칼럼의 각 엘리먼트를 검사하여 대괄호([])를 포함하는 경우 출력:")
# for idx, cell in df[last_col].items():
#     try:
#         # 문자열 형태의 리스트를 실제 리스트로 변환
#         cell_list = ast.literal_eval(cell)
#         if isinstance(cell_list, list):
#             for element in cell_list:
#                 if isinstance(element, str) and re.search(r'\[.*?\]', element):
#                     print(f"Index {idx}: {element}")
#     except Exception as e:
#         #print(f"Index {idx} 변환 실패: {cell} (오류: {e})")
#         pass


# --- 4. 처리된 DataFrame을 새로운 CSV 파일로 저장 ---
output_path = "german-5_processed.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\n처리된 DataFrame이 '{output_path}' 파일로 저장되었습니다.")

변환 실패: x (오류: malformed node or string: <_ast.Name object at 0x7fd5437b86d0>)
변환 실패: x (오류: malformed node or string: <_ast.Name object at 0x7fd5945c35e0>)
변환 실패: [Anfang vierzig und keinen Lebensgefährten; sieht so aus, als hätte sie über ihre Karriere den Anschluss verpasst.'] (오류: invalid syntax (<unknown>, line 1))
변환 실패: x (오류: malformed node or string: <_ast.Name object at 0x7fd542c86c10>)
변환 실패: [Vielen Eltern, die schon ein Kind haben, gehen die Augen über, wenn das nächste kommt: Das ältere wächst über Nacht.'] (오류: invalid syntax (<unknown>, line 1))
변환 실패: ['Dieser Lärm ist ja nicht zum Aushalten! Es ist nicht zum Aushalten, was der Mensch für einen Blödsinn von sich gibt!', 'Er stank auf zwei Meter so, dass es nicht zum Aushalten war.] (오류: EOL while scanning string literal (<unknown>, line 1))
변환 실패: x (오류: malformed node or string: <_ast.Name object at 0x7fd542c754f0>)
변환 실패: nan (오류: malformed node or string: <_ast.Name object at 0x7fd5945c3fd0>)
변환 실패: x (오류: malformed 

콜론처리

In [24]:
import pandas as pd

file_path = "german-5.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 첫 번째 칼럼 이름 가져오기
first_col = df.columns[0]

# 첫 번째 칼럼의 값에 콜론이 포함된 행 전체를 필터링 (na=False 옵션 추가)
filtered_df = df[df[first_col].astype(str).str.contains(";", na=False)]

filtered_df

,Redewendung,Bedeutung,Beispiele
44,Ablage rund; Ablage P,Papierkorb,['Die Businesspläne landeten ungelesen in der ...
55,sich auf dem Absatz umdrehen/ umwenden; auf de...,spontan umkehren,['Als er unter den Gästen seine frühere Frau e...
86,mit vielem Ach und Weh; mit/unter Weh und Ach,"mit vielem Klagen, Stöhnen; höchst ungern",['Mit vielem Ach und Weh hatte sie endlich die...
93,etwas außer Acht lassen; etwas aus der/aus all...,etwas nicht beachten,['Bei der Entscheidung hatte man wichtige Aspe...
116,an die falsche/unrechte/verkehrte Adresse komm...,bei jemandem auf eine den positiven Erwartunge...,"['Dass eins klar ist, Herr Schenk. Wenn Sie ’n..."
...,...,...,...
10229,der getretene Wurm krümmt sich; auch der Wurm ...,"auch der noch so Unterlegene, Ängstliche, Unte...","['Glauben Sie nicht, dass ich weiterhin so mit..."
10283,lange Zähne machen; mit langen Zähnen essen,"auffällig langsam kauen und damit zeigen, dass...","['Wenn es Hafergrütze gab, machten die Kinder ..."
10284,die Zähne zusammenbeißen; auf die Zähne beißen,"Schmerzen, schwere Zeiten, Unangenehmes tapfer...",['Mühsam schiebt sie den Rollator ins Wohnzimm...
10374,für alle Zeiten; für Zeit und Ewigkeit,für immer,['Eine Koalition ist nicht als ein Bündnis für...


In [41]:
import pandas as pd
import ast

# CSV 파일 읽기 (파일 경로 및 인코딩을 필요에 맞게 조정)
file_path = "german-5.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 마지막 칼럼 이름 가져오기
last_col = df.columns[-1]

threshold = 0
threshold_m = 20

print("마지막 칼럼의 리스트 내에서 길이가 너무 짧은 엘리먼트들:")

# 각 셀을 순회하면서
for idx, cell in df[last_col].items():
    try:
        # 셀이 문자열로 저장된 리스트라면 실제 리스트로 변환
        lst = ast.literal_eval(cell)
        if isinstance(lst, list):
            for elem in lst:
                # elem이 문자열인 경우 앞뒤 공백 제거 후 길이 검사
                if isinstance(elem, str) and len(elem.strip()) > threshold and len(elem.strip()) < threshold_m:
                    print(f"Index {idx}: {elem.strip()}")
    except Exception as e:
        #print(f"Index {idx} 변환 실패: {cell} (오류: {e})")
        pass

마지막 칼럼의 리스트 내에서 길이가 너무 짧은 엘리먼트들:
Index 1558: Er ist Major a. D.
Index 2194: Wir sind fertig.


In [36]:
import pandas as pd

# CSV 파일 읽기
file_path = "german-5.csv"
df = pd.read_csv(file_path, encoding="utf-8-sig")

# 첫 번째 칼럼 이름 가져오기
first_col = df.columns[0]

# 첫 번째 칼럼의 문자열 길이가 80보다 크고 91보다 작은 행 필터링
filtered_df = df[(df[first_col].astype(str).str.len() > 60) & (df[first_col].astype(str).str.len() < 80)]

print("첫 번째 칼럼에서 길이가 80보다 크고 91보다 작은 행들:")
print(filtered_df[first_col])

첫 번째 칼럼에서 길이가 80보다 크고 91보다 작은 행들:
46       an jemandem ablaufen wie das Wasser am Entenfl...
55       sich auf dem Absatz umdrehen/umwenden; auf dem...
61       dreimal/zweimal abgeschnitten/abgesägt und imm...
137      von Ackerbau und Viehzucht/von Tuten und Blase...
172      jemanden, etwas auf dem Altar der Freundschaft...
                               ...                        
10385    jemand hat/jemand besitzt/in jemandem steckt d...
10421    jemandem Zucker/Puderzucker/Staubzucker in den...
10422    jemandem Zucker/Puderzucker/Staubzucker in den...
10428    was du nicht willst, dass man dir tu, das füg ...
10508    dann/sonst kannst du dich zusammenfegen/zusamm...
Name: Redewendung, Length: 206, dtype: object


ver 6. 칼럼위치 변경

In [1]:
import pandas as pd

# CSV 파일 읽기
file_path = "german-6.csv"  
df = pd.read_csv(file_path)

# 마지막 컬럼을 첫 번째로 이동
last_column = df.columns[-1]  # 마지막 컬럼명 가져오기
df = df[[last_column] + [col for col in df.columns[:-1]]]  # 재정렬

df.to_csv("german-6_reordered.csv", index=False)  # 새 파일로 저장

print("마지막 컬럼이 첫 번째로 이동된 파일이 'german-6_reordered.csv'로 저장되었습니다.")

마지막 컬럼이 첫 번째로 이동된 파일이 'german-6_reordered.csv'로 저장되었습니다.


In [2]:
second_column = df.columns[1]

# "/"가 포함된 행 필터링
filtered_df = df[df[second_column].astype(str).str.contains("/", na=False)]

# 결과 저장
output_path = "german-6_sample.csv"
filtered_df.to_csv(output_path, index=False)

인칭대명사 이슈

In [4]:
import pandas as pd
from collections import Counter

# CSV 파일 읽기
df = pd.read_csv("german-5.csv")

# 첫 번째 칼럼 선택 (칼럼 이름이 없을 경우 index 사용)
first_column = df.iloc[:, 0].astype(str)

# 찾을 단어 리스트
search_terms = [
    "dich/", "/dich", "dir/", "/dir", "mich/", "/mich", "mir/", "/mir",
    "ihm/", "/ihm", "ihr/", "/ihr", "sie/", "/sie", "ihn/", "/ihn",
    "uns/", "/uns", "euch/", "/euch", "ihnen/", "/ihnen"
]

# 단어 등장 횟수 카운트
word_counts = {word: first_column.str.count(fr"\b{word}\b").sum() for word in search_terms}

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(word_counts.items(), columns=["Word", "Count"])

# 정렬 (내림차순)
result_df = result_df.sort_values(by="Count", ascending=False)

# 데이터 표시
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Word Count Result", dataframe=result_df)

Word Count Result


In [5]:
import pandas as pd
from collections import Counter

# CSV 파일 읽기
df = pd.read_csv("german-5.csv")

# 첫 번째 칼럼 선택 (칼럼 이름이 없을 경우 index 사용)
first_column = df.iloc[:, 0].astype(str)

# 찾을 단어 리스트
search_terms = [
    "dich/", "/dich", "dir/", "/dir", "mich/", "/mich", "mir/", "/mir",
    "ihm/", "/ihm", "ihr/", "/ihr", "sie/", "/sie", "ihn/", "/ihn",
    "uns/", "/uns", "euch/", "/euch", "ihnen/", "/ihnen"
]

# 단어 등장 횟수 카운트
word_counts = Counter()

for word in search_terms:
    word_counts[word] = first_column.str.contains(fr"\b{word}\b", regex=True).sum()

# 결과 출력
for word, count in word_counts.items():
    print(f"{word}: {count}")


dich/: 5
/dich: 1
dir/: 7
/dir: 1
mich/: 3
/mich: 0
mir/: 4
/mir: 3
ihm/: 0
/ihm: 3
ihr/: 1
/ihr: 6
sie/: 1
/sie: 4
ihn/: 2
/ihn: 1
uns/: 1
/uns: 2
euch/: 0
/euch: 3
ihnen/: 0
/ihnen: 0


In [7]:
import pandas as pd
from collections import Counter

# CSV 파일 읽기
df = pd.read_csv("german-5.csv")

# 첫 번째 칼럼 선택 (칼럼 이름이 없을 경우 index 사용)
first_column = df.iloc[:, 0].astype(str)

# 찾을 단어 리스트
search_terms = [
    "dich/", "/dich", "dir/", "/dir", "mich/", "/mich", "mir/", "/mir",
    "ihm/", "/ihm", "ihr/", "/ihr", "sie/", "/sie", "ihn/", "/ihn",
    "uns/", "/uns", "euch/", "/euch", "ihnen/", "/ihnen"
]

# 단어 등장 횟수 카운트
word_counts = {word: first_column.str.count(fr"\b{word}\b").sum() for word in search_terms}

# 총합 계산
total_count = sum(word_counts.values())

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(word_counts.items(), columns=["Word", "Count"])

# 총합 추가
result_df.loc[len(result_df)] = ["TOTAL", total_count]

# 정렬 (내림차순)
result_df = result_df.sort_values(by="Count", ascending=False)

tools.display_dataframe_to_user(name="Word Count with Total", dataframe=result_df)


Word Count with Total


/처리 끝나고 jemanden, etwas 처리하는거

In [1]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-6_master.csv")

# 첫 번째 칼럼에서 'jemanden, etwas' 또는 'jemandem, etwas'가 포함된 행 찾기
filtered_rows = df[df.iloc[:, 0].str.contains(r'\bjemanden, etwas\b|\bjemandem, etwas\b', na=False, regex=True)]

# 행 번호 추가해서 출력
for index, row in filtered_rows.iterrows():
    print(f"Row {index}: {row.iloc[0]}")

Row 173: ['jemanden, etwas auf dem Altar der Freundschaft opfern', 'jemanden, etwas auf dem Altar der Liebe opfern', 'jemanden, etwas auf dem Altar des Vaterlandes opfern']
Row 247: ['jemanden, etwas auf die Anklagebank bringen', 'jemanden, etwas auf die Anklagebank setzen']
Row 315: ['von jemandem, etwas angezogen werden wie die Motten vom Licht']
Row 525: ['sich nach jemandem, etwas die Augen ausgucken', 'sich nach jemandem, etwas die Augen aus dem Kopf gucken', 'sich nach jemandem, etwas die Augen aus dem Kopf schauen']
Row 534: ['kein Auge von jemandem, etwas lassen', 'kein Auge von jemandem, etwas wenden']
Row 540: ['jemanden, etwas nicht aus dem Auge lassen', 'jemanden, etwas nicht aus den Augen lassen']
Row 542: ['jemanden, etwas aus dem Auge verlieren', 'jemanden, etwas aus den Augen verlieren']
Row 547: ['jemanden, etwas im Auge behalten', 'jemanden, etwas im Auge haben']
Row 564: ['jemanden, etwas mit anderen Augen sehen', 'jemanden, etwas mit neuen Augen sehen']
Row 581: ['s

In [2]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-6_master.csv")

# 첫 번째 칼럼에서 'jemanden, etwas' 또는 'jemandem, etwas'가 포함된 행 찾기
filtered_rows = df[df.iloc[:, 0].str.contains(r'\bgegen\b', na=False, regex=True)]

# 행 번호 추가해서 출력
for index, row in filtered_rows.iterrows():
    print(f"Row {index}: {row.iloc[0]}")

Row 300: ['gegen jemanden nicht anstinken können', 'gegen etwas nicht anstinken können']
Row 449: ['eine Attacke gegen jemanden reiten', 'eine Attacke gegen etwas reiten']
Row 747: ['gegen bar']
Row 1720: ['gegen Dummheit kämpfen Götter selbst vergebens']
Row 2180: ['gegen jemanden, etwas zu Felde ziehen']
Row 2505: ['Front gegen jemanden machen', 'Front gegen etwas machen']
Row 3065: ['Gold gegen etwas sein', 'Gold gegen jemanden sein']
Row 3297: ['etwas gegen jemanden haben', 'etwas gegen etwas haben']
Row 3441: ['die Hand gegen jemanden erheben']
Row 3517: ['etwas gegen jemanden in der Hand haben', 'etwas gegen jemanden in Händen haben']
Row 4709: ['gegen jemanden, etwas nicht anstinken können, sich etwas am Arsch abfingern können die Konsequenz ziehen', 'gegen jemanden, etwas nicht anstinken können, sich etwas am Arsch abfingern können die Konsequenzen ziehen']
Row 4883: ['gegen jemanden, etwas ist kein Kraut gewachsen']
Row 5724: ['drei Meilen gegen den Wind', 'sieben Meilen gegen

In [9]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-6_master.csv")

filtered_rows = df[df.iloc[:, 0].str.contains(r'\bdritte\b', na=False, regex=True)]

# 행 번호 추가해서 출력
for index, row in filtered_rows.iterrows():
    print(f"Row {index}: {row.iloc[0]}")

Row 2887: ['das dritte Geschlecht']
Row 10338: ['dritte Zähne']


In [15]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 마지막 칼럼 선택 후 길이가 5 이하인 행 필터링
filtered_rows = df[df.iloc[:, -1].astype(str).str.len() <= 5]

# 행 번호와 함께 출력
for index, row in filtered_rows.iterrows():
    print(f"Row {index}: {row.iloc[-1]}")

Row 26: []
Row 115: []
Row 236: []
Row 237: []
Row 347: []
Row 679: []
Row 1138: []
Row 1149: []
Row 1190: []
Row 1301: []
Row 1342: []
Row 1492: []
Row 1540: []
Row 1646: []
Row 1800: []
Row 1908: []
Row 1919: []
Row 2017: []
Row 2250: []
Row 2298: []
Row 2970: []
Row 2977: []
Row 3225: []
Row 3425: []
Row 3766: []
Row 3924: []
Row 3986: []
Row 4044: []
Row 4167: []
Row 4686: []
Row 4693: []
Row 4727: []
Row 4778: []
Row 4874: []
Row 5018: []
Row 5371: []
Row 5374: []
Row 5461: []
Row 5462: []
Row 5516: []
Row 5517: []
Row 5538: []
Row 5576: []
Row 5685: []
Row 5742: []
Row 5790: []
Row 5837: []
Row 5871: []
Row 5872: []
Row 5895: []
Row 5896: []
Row 5952: []
Row 6050: []
Row 6268: []
Row 6350: []
Row 6356: []
Row 6428: []
Row 6429: []
Row 6431: []
Row 6432: []
Row 6433: []
Row 6434: []
Row 6435: []
Row 6436: []
Row 6530: []
Row 6531: []
Row 6610: []
Row 6689: []
Row 6695: []
Row 6727: []
Row 6752: []
Row 6840: []
Row 6845: []
Row 6848: []
Row 6855: []
Row 6905: []
Row 6917: []
Row 69

In [12]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-6_master.csv")

# 마지막 칼럼 선택 후 길이가 5 이하인 값들을 "[]"로 변경
df.iloc[:, -1] = df.iloc[:, -1].astype(str).apply(lambda x: "[]" if len(x) <= 5 else x)

# 필요하면 변경된 CSV 파일 저장
df.to_csv("german-7.csv", index=False)

In [3]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 각 행(row)의 모든 열을 문자열로 변환하고, 해당 행의 전체 길이 계산
row_lengths = df.astype(str).apply(lambda x: ''.join(x), axis=1).str.len()

# 두 번째 칼럼의 길이 계산
first_column_lengths = df.iloc[:, 1].astype(str).str.len()

# 두 번째 칼럼에 "/" 또는 ";"가 포함되어 있으면 제외
second_column_no_slash_or_semicolon = ~df.iloc[:, 1].astype(str).str.contains(r"[\/;]", regex=True)

# 특정 길이 이상인 행을 필터링 (전체 행 길이와 두 번째 칼럼 길이 모두 고려)
threshold = 400  # 원하는 행 전체 길이 기준
first_column_threshold = 60  # 두 번째 칼럼 최소 길이 기준

filtered_rows = df[(row_lengths >= threshold) & 
                   (first_column_lengths >= first_column_threshold) & 
                   second_column_no_slash_or_semicolon]

# 행 번호와 함께 출력
for index, row in filtered_rows.iterrows():
    print(f"Row {index}: {row.to_list()}")

Row 315: ["['von jemandem, etwas angezogen werden wie die Motten vom Licht']", 'von jemandem, etwas angezogen werden wie die Motten vom Licht', 'unwiderstehlich in den Bann einer Person, Sache gezogen werden', "['Frauen werden von ihm angezogen wie die Motten vom Licht', 'Durch so etwas wird die Presse natürlich angezogen wie die Motten vom Licht – quotenhungrige Moderatoren gieren nach psychopathischen Killern', 'Von Unternehmen mit einer großen Gewinndynamik werden sie angezogen wie die Motten vom Licht']"]
Row 927: ["['wenn der Berg nicht zum Propheten kommt, muss der Prophet zum Berg gehen']", 'wenn der Berg nicht zum Propheten kommt, muss der Prophet zum Berg gehen', 'einer muss den ersten Schritt tun, man kann nicht darauf warten, dass jemand, etwas den eigenen Wünschen entgegenkommt, sondern man muss die Anstrengung machen, sich selbst zu helfen', "['Der Streit ist weiß Gott keinen Freundschaftsbruch wert. Wenn sie hartnäckig bleibt, dann muss der Prophet halt zum Berg gehen, sa

In [1]:
import pandas as pd
import ast  # 문자열을 실제 리스트로 변환하는데 사용

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 첫 번째 칼럼이 리스트 형태이고, 원소가 2개 이상인지 확인하는 함수
def is_multiple_values(value):
    value = str(value).strip()

    # 리스트 형태인지 체크하고, 실제 리스트로 변환 시도
    if value.startswith("[") and value.endswith("]"):
        try:
            parsed_value = ast.literal_eval(value)  # 문자열을 실제 리스트로 변환
            if isinstance(parsed_value, list) and len(parsed_value) > 1:
                return True
        except (SyntaxError, ValueError):
            return False

    return False

# 첫 번째 칼럼이 리스트이고, 원소가 2개 이상인지 확인
first_column_multiple = df.iloc[:, 0].apply(is_multiple_values)

# 두 번째 칼럼에 "/"가 없는 경우 확인
second_column_no_slash = ~df.iloc[:, 1].astype(str).str.contains("/", regex=False)

# 두 번째 칼럼에 'jemand'와 'etwas'가 둘 다 포함된 경우 제외
def contains_both_keywords(value):
    value = str(value).lower()  # 대소문자 무시
    return ("jemand" in value and "etwas" in value) or ("jemand" in value and "sache" in value) or ";" in value

second_column_no_both_keywords = ~df.iloc[:, 1].apply(contains_both_keywords)

# 조건을 만족하는 행 필터링
filtered_rows = df[first_column_multiple & second_column_no_slash & second_column_no_both_keywords]

# 행 번호와 함께 출력
for index, row in filtered_rows.iterrows():
    print(f"Row {index}: {row.to_list()}")

Row 289: ["['das ist Ansichtssache', 'etwas ist Ansichtssache']", 'das, etwas ist Ansichtssache', 'darüber, über etwas kann man verschiedener Meinung sein', "['Wie man darauf reagiert, das ist doch wohl Ansichtssache.', 'Die Art der Kopfbedeckung ist Ansichtssache.']"]
Row 438: ["['einen langen Atem haben', 'den längeren Atem haben']", 'einen langen Atem haben, den längeren Atem haben', 'es bei einer Auseinandersetzung, einem Wettstreit lange, länger als der Gegner aushalten', "['Wir wollen doch mal sehen, wer von uns beiden den längeren Atem hat. Die Guerillas haben einen langen Atem.', 'Er besaß einen langen Atem im Kampf.']"]
Row 635: ["['mit jemandem ausgeredet haben', 'miteinander ausgeredet haben']", 'mit jemandem, miteinander ausgeredet haben', 'an jemandem, aneinander nicht mehr interessiert sein', "['Die beiden haben offensichtlich miteinander ausgeredet; sie grüßen sich nicht einmal mehr.']"]
Row 736: ["['mit harten Bandagen kämpfen', 'mit harten Bandagen ringen']", 'mit hart

칼럼 비워있는애들

In [5]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 하나라도 비어있는(누락된) 값이 있는 행 찾기
missing_values_rows = df[df.isnull().any(axis=1) | (df.astype(str) == '').any(axis=1)]

# 행 번호와 함께 출력
for index, row in missing_values_rows.iterrows():
    print(f"Row {index}: {row.to_list()}")

키워드 찾아내기.. n낱말 이상 2번이상반복되면 확인

In [13]:
import pandas as pd
import ast  # 문자열을 실제 리스트로 변환하는데 사용

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 마지막 칼럼을 리스트로 변환하고, 단어 개수를 세는 함수
def count_words_in_list(value):
    try:
        parsed_value = ast.literal_eval(value)  # 문자열을 실제 리스트로 변환
        if isinstance(parsed_value, list):  # 리스트인지 확인
            word_count = sum(len(str(item).split()) for item in parsed_value)  # 리스트 내부 모든 단어 개수 합산
            return word_count if word_count > 0 else float('inf')  # 단어 개수가 0이면 제외
    except (SyntaxError, ValueError):
        return float('inf')  # 변환 실패하면 제외
    return float('inf')

# 마지막 칼럼의 단어 개수 계산
df['word_count'] = df.iloc[:, -1].astype(str).apply(count_words_in_list)

# 연속으로 10단어 이하인 행이 2번 이상 있는 경우 찾기
consecutive_count = 0  # 연속된 행 개수 카운트
threshold = 10 # 단어 개수 기준

for index in range(len(df) - 1):
    if df.loc[index, 'word_count'] <= threshold:
        consecutive_count += 1
    else:
        consecutive_count = 0  # 연속 끊기면 초기화
    
    if consecutive_count >= 2:
        print(f"Row {index}: {df.iloc[index].to_list()}")

Row 12: ["['sich einen abasten']", 'sich einen abasten', 'sich mit etwas Schwerem abplagen', "['Ich habe mir mit der Kiste ganz schön einen abgeastet.']", 10.0]
Row 50: ["['auf etwas abonniert sein']", 'auf etwas abonniert sein', 'etwas abonniert haben', "['Ich bin seit zwei Jahren auf diese Zeitschrift abonniert.']", 9.0]
Row 180: ["['vor alters']", 'vor alters', 'vor langer Zeit, einstmals', "['Vor alters stand dort eine Burg.']", 6.0]
Row 200: ["['sich einen andudeln']", 'sich einen andudeln', 'sich betrinken', "['Sie hat sich ordentlich einen mit Sekt angedudelt.']", 8.0]
Row 257: ["['das kannst du annehmen!']", 'das kannst du annehmen!', 'das ist sicher!', "['Dem werd ich die Meinung geigen, das kannst du annehmen!']", 10.0]
Row 378: ["['ein Armutszeugnis sein']", 'ein Armutszeugnis sein', 'der Nachweis für jemandes Unfähigkeit, Unvermögen sein', "['Es wäre ja ein Armutszeugnis, wenn wir das nicht hinkriegten.']", 10.0]
Row 422: ["['nach Art des Hauses']", 'nach Art des Hauses', '

; 전처리

In [14]:
import pandas as pd
import ast  # 문자열을 실제 리스트로 변환하는데 사용

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 첫 번째 칼럼이 리스트이고, 그 안에 ";"가 포함된 경우 찾기
def contains_semicolon(value):
    try:
        parsed_value = ast.literal_eval(value)  # 문자열을 실제 리스트로 변환
        if isinstance(parsed_value, list):  # 리스트인지 확인
            return any(";" in str(item) for item in parsed_value)  # 리스트 내부 요소 중 ";" 포함 여부 확인
    except (SyntaxError, ValueError):
        return False  # 변환 실패하면 False 반환
    return False

# 첫 번째 칼럼에서 ";"가 포함된 행 찾기
semicolon_rows = df[df.iloc[:, 0].astype(str).apply(contains_semicolon)]

# 행 번호와 함께 출력
for index, row in semicolon_rows.iterrows():
    print(f"Row {index}: {row.to_list()}")

Row 86: ["['mit vielem Ach und Weh; mit Weh und Ach', 'mit vielem Ach und Weh; unter Weh und Ach']", 'mit vielem Ach und Weh; mit/unter Weh und Ach', 'mit vielem Klagen, Stöhnen; höchst ungern', "['Mit vielem Ach und Weh hatte sie endlich die zwanzig Euro aus ihrem Portemonnaie gekramt. Dach.']"]
Row 487: ["['das ist ein Aufwasch; das geht in einem Aufwasch', 'das ist ein Aufwasch; das geht in einem Abwasch', 'das ist ein Aufwasch; das machen wir in einem Aufwasch', 'das ist ein Aufwasch; das machen wir in einem Abwasch', 'das ist ein Abwasch; das geht in einem Aufwasch', 'das ist ein Abwasch; das geht in einem Abwasch', 'das ist ein Abwasch; das machen wir in einem Aufwasch', 'das ist ein Abwasch; das machen wir in einem Abwasch']", 'das ist ein Aufwasch/Abwasch; das geht/das machen wir in einem Aufwasch/Abwasch', 'das lässt sich alles zusammen erledigen', "['Wenn ich den Koffer aufgebe, kann ich auch gleich die Fahrkarten besorgen, das ist dann ein Aufwasch.', 'Finanziell besteht als

In [1]:
import pandas as pd
import ast  # 문자열을 리스트로 변환하는데 사용
import re   # 정규식 사용

# CSV 파일 로드
df = pd.read_csv("german-7.csv")

# 특정 패턴(`. + 띄어쓰기 + 10자 이하 단어 + . + '`)을 찾는 함수
def contains_specific_pattern(value):
    try:
        # 문자열이 리스트 형태로 저장되어 있다면 변환
        if isinstance(value, str) and value.startswith("[") and value.endswith("]"):
            parsed_value = ast.literal_eval(value)  # 문자열을 리스트로 변환
        else:
            parsed_value = value  # 이미 리스트라면 그대로 사용
        
        # 리스트인지 확인하고, 각 문장을 검사
        if isinstance(parsed_value, list) and len(parsed_value) > 0:
            for sentence in parsed_value:
                sentence = str(sentence).strip()  # 공백 제거
                
                # 정규식: `. + 띄어쓰기 + 10자 이하 단어 + . + '`
                matches = re.findall(r'\.\s(\w{1,10})\.\'?', sentence)
                
                if matches:
                    return True  # 하나라도 조건을 만족하면 True 반환
    except (SyntaxError, ValueError):
        return False  # 변환 실패하면 False 반환
    return False

# 마지막 칼럼에서 조건을 만족하는 행 찾기
filtered_rows = df[df.iloc[:, -1].apply(contains_specific_pattern)]

# 행 번호와 함께 출력
if not filtered_rows.empty:
    for index, row in filtered_rows.iterrows():
        print(f"Row {index}: {row.to_list()}")
else:
    print("조건을 만족하는 행이 없습니다.")

Row 189: ["['von Amts wegen']", 'von Amts wegen', 'in behördlichem Auftrag', "['Das baufällige Gebäude wurde von Amts wegen abgebrochen. Fräulein.']"]
Row 428: ["['ein Ass aus dem Ärmel ziehen']", 'ein Ass aus dem Ärmel ziehen', 'für ein Problem eine überraschende Lösung finden', "['Als alle das Spiel schon verloren glaubten, zog der Trainer noch ein Ass aus dem Ärmel. Aas. Viele geläufige Redewendungen haben ihren Ursprung im', 'So kann es sein, dass das Gegenüber, das nicht ganz fair ist, mit gezinkten']"]
Row 619: ["['es hängt jemandem aus']", 'es hängt jemandem aus', 'jemand verliert die Nerven', "['und plötzlich hängt’s uns aus und wir schlagen drein? Nur ja nicht. Kreuz.']"]
Row 645: ["['sehe ich so aus?', 'sehe ich danach aus?']", 'sehe ich so/danach aus?', 'kann man das von mir glauben?', "['Und ich soll ihn geschlagen haben, sehe ich so vielleicht aus? Sicher haben Sie selbst eine Frau. Nein. Sehe ich danach aus?']"]
Row 1574: ["['außer Dienst']", 'außer Dienst', '(in Bezug au